In [1]:
from kafka import KafkaAdminClient
from kafka.protocol.group import MemberAssignment
from kafka import KafkaConsumer, TopicPartition
import pandas as pd

In [2]:
group_id = 'reco-worker-oplog'
topic = 'oplog'

In [3]:
result = []
bootstrap_servers=[f'cdh-work-0{i:02}:9092' for i in range(1, 11)]
admin_client = KafkaAdminClient(bootstrap_servers=bootstrap_servers)
consumer = KafkaConsumer(topic,
                         bootstrap_servers=bootstrap_servers,
                         group_id=group_id)
for member in admin_client.describe_consumer_groups([group_id])[0][5]:
    for data in MemberAssignment.decode(member[4]).assignment:
        topic = data[0]
        parts = data[1]
        for part in parts:
            topart = TopicPartition(topic, part)
            last_offsets = consumer.end_offsets([topart])
            committed = consumer.committed(topart)
            if last_offsets is not None and committed is not None:
                offset = last_offsets.get(topart)
                lag = offset - committed
                result.append({
                    'topic': topic,
                    'group': group_id,
                    'partition': part,
                    'committed': committed,
                    'current_offset': offset,
                    'lag': lag
                })

In [4]:
df = pd.DataFrame(result)
display(df.head())
display(df[['lag']].describe())

,topic,group,partition,committed,current_offset,lag
0,oplog,reco-worker-oplog,194,615301,615457,156
1,oplog,reco-worker-oplog,204,616021,616166,145
2,oplog,reco-worker-oplog,82,615134,615298,164
3,oplog,reco-worker-oplog,143,616580,616654,74
4,oplog,reco-worker-oplog,185,615489,615637,148


,lag
count,240.000000
mean,270.341667
std,582.227256
min,0.000000
25%,87.000000
50%,120.000000
75%,165.250000
max,2821.000000
